In [17]:
import carla
import random
import math
from collections import defaultdict

# Tolleranza per considerare i waypoints sullo stesso asse X o Y
tolerance = 2

def p_wp(w):
  return f"({round(w.transform.location.x,3)}) - ({round(w.transform.location.y,3)})"

#lista di adiacenza di coppie di waypoints avendi il secondo waypoint in comune
#significa che hanno punti di partenza differenti ma si intersecano nel secondo waypoint 
def group_pairs_by_second_waypoint(waypoint_pairs):
    
    # Dizionario (mappa) per raggruppare le coppie in base alle coordinate (x, y) del secondo waypoint di ciascuna coppia
    coord_groups = defaultdict(list)
    
    # Raggruppa le coppie in base alle coordinate x e y del secondo waypoint
    for wp1, wp2 in waypoint_pairs:
        #la chiave è rappresentata dalle coordinate del secondo wp
        coord_wp2 = (round(wp2.transform.location.x, 3), round(wp2.transform.location.y, 3))
        coord_groups[coord_wp2].append((wp1, wp2))
    
    # Stampa i gruppi di coppie per ciascun secondo waypoint
    for wp2_coords, pairs in coord_groups.items():
        print(f"le coordinate {wp2_coords} accomunano le seguenti coppie:")
        for pair in pairs:
            print(p_wp(pair[0]) + ", " + p_wp(pair[1]))
        print()

#colora il percorso tra due waypoint, bianco se i punto sono allineati sulla stessa trada, rosso se c'è una curva
def draw_route(wp1, wp2, symbol, r, g, b):
    distance = 3.0  # Distanza tra i waypoint intermedi in metri
    symbol = str(symbol)

    wp1 = wp1.next(distance)[0]
    
    # Disegna simboli lungo il percorso fino al waypoint di destinazione
    while wp1.transform.location.distance(wp2.transform.location) > distance:
        # Disegna il simbolo nel waypoint corrente
        world.debug.draw_string(wp1.transform.location, symbol, draw_shadow=False,
                                color=carla.Color(r, g, b), life_time=60.0, persistent_lines=True)
        # Vai al prossimo waypoint
        wp1 = wp1.next(distance)[0]


#disegna due x rosse sui wp
def draw_couple(idx, wp1, wp2):
    # Disegna una 'X' su wp1 con il numero progressivo
    world.debug.draw_string(wp1.transform.location, f"X {idx}", 
                            draw_shadow=True, 
                            color=carla.Color(r=0, g=255, b=0), 
                            life_time=60.0, 
                            persistent_lines=True)

    # Disegna una 'X' su wp2 con lo stesso numero progressivo
    world.debug.draw_string(wp2.transform.location, f"X {idx}", 
                            draw_shadow=True, 
                            color=carla.Color(r=255, g=0, b=0), 
                            life_time=60.0, 
                            persistent_lines=True)


#unused: Funzione per calcolare l'angolo tra due waypoints di Carla
def calculate_angle(waypoint1, waypoint2):
    # Ottieni le coordinate x e y dei due waypoints
    dx = waypoint2.transform.location.x - waypoint1.transform.location.x
    dy = waypoint2.transform.location.y - waypoint1.transform.location.y
    # Calcola l'angolo in gradi
    angle = math.degrees(math.atan2(dy, dx))
    return abs(angle)

# Funzione checonfronta ciascuna coppia di wp
def analyze_waypoints(pairs):
    
    for idx, (wp1, wp2) in enumerate(pairs):
        idx = idx + 1
        # Ottieni le coordinate x e y dei waypoints
        x1, y1 = wp1.transform.location.x, wp1.transform.location.y
        x2, y2 = wp2.transform.location.x, wp2.transform.location.y

        dx = abs(x1 - x2)
        dy = abs(y1 - y2)
        
        # Controllo se sono sullo stesso asse x o y
        if dx <= tolerance or  dy <= tolerance:
            print(f"Pair {idx}: I waypoints sono allineati ({wp1.transform.location.x} - {wp1.transform.location.y}) - ({wp2.transform.location.x} - {wp2.transform.location.y})")
            draw_couple(idx, wp1, wp2)
            draw_route( wp1, wp2, idx, 0, 0, 255)
        else:
            # altrimenti c'è un angolo tra i due wp
            print(f"Pair {idx}: tra i waypoints c'è una curva ({wp1.transform.location.x} - {wp1.transform.location.y}) - ({wp2.transform.location.x} - {wp2.transform.location.y})")
            draw_couple(idx, wp1, wp2)
            draw_route(wp1, wp2, idx, 255, 255, 255 )
            
            '''if 30 <= angle <= 90:
                draw_couple(idx, wp1, wp2)
                print(f"Pair {idx}: L'angolo tra i waypoints è di {angle:.2f} gradi (tra 30° e 90°). {wp1.transform.location} - {wp2.transform.location}")
                draw_route(wp1, wp2, idx )
            else:
                draw_couple(idx, wp1, wp2)
                print(f"Pair {idx}: L'angolo tra i waypoints è di {angle:.2f} gradi (non tra 30° e 90°). {wp1.transform.location} - {wp2.transform.location}")
                draw_route(wp1, wp2, idx) '''



# Connessione a Carla
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()
bp_lib = world.get_blueprint_library() 
map = world.get_map()

# Trova un incrocio casuale
spawn_points = map.get_spawn_points()
random_point = random.choice(spawn_points)
waypoint = map.get_waypoint(random_point.location)

# Cerca un incrocio a caso nella mappa a partire da un pnto di spawn a caso
while waypoint.get_junction() is None:
    random_point = random.choice(spawn_points)
    waypoint = map.get_waypoint(random_point.location)

#ritorna la lista di coppie di wp dell'intersezione dove il primo membro di ciascuna coppia è quello di partenza
wp_pairs = waypoint.get_junction().get_waypoints(carla.LaneType.Driving)


# Posiziona la telecamera del simulatore sopra l'incrocio
spectator = world.get_spectator()
camera_location = carla.Location(x=waypoint.transform.location.x, y=waypoint.transform.location.y, z=60) #altezza 60
camera_rotation = carla.Rotation(pitch=-90)  # Orientamento verso il basso
spectator.set_transform(carla.Transform(camera_location, camera_rotation))


########fine main code #####################

# Analizza la lista delle coppie di waypoints
analyze_waypoints(wp_pairs)
group_pairs_by_second_waypoint(wp_pairs)


Pair 1: I waypoints sono allineati (27.137821197509766 - 69.78325653076172) - (58.047794342041016 - 69.8227767944336)
Pair 2: I waypoints sono allineati (58.05227279663086 - 66.3227767944336) - (27.142295837402344 - 66.28325653076172)
Pair 3: tra i waypoints c'è una curva (40.52924728393555 - 52.59458541870117) - (58.047794342041016 - 69.8227767944336)
Pair 4: tra i waypoints c'è una curva (58.05227279663086 - 66.3227767944336) - (44.02894973754883 - 52.548702239990234)
Pair 5: tra i waypoints c'è una curva (27.137819290161133 - 69.78325653076172) - (44.02894973754883 - 52.548702239990234)
Pair 6: tra i waypoints c'è una curva (40.52924728393555 - 52.59458541870117) - (27.14229393005371 - 66.28325653076172)
le coordinate (58.048, 69.823) accomunano le seguenti coppie:
(27.138) - (69.783), (58.048) - (69.823)
(40.529) - (52.595), (58.048) - (69.823)

le coordinate (27.142, 66.283) accomunano le seguenti coppie:
(58.052) - (66.323), (27.142) - (66.283)
(40.529) - (52.595), (27.142) - (66